In [1]:
import pandas as pd
import numpy as np
from path import Path

In [2]:
# Read the raw us cities data file
file_path = Path("../../data/raw/uscities.csv")
city_df = pd.read_csv(file_path)
print(city_df.shape)
city_df.head()


(28372, 17)


,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574.0,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7839,-80.2102,6445545,5019.0,polygon,False,True,America/New_York,1,33129 33125 33126 33127 33128 33149 33144 3314...,1840015149
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7936,-96.7662,5743938,1526.0,polygon,False,True,America/Chicago,1,75287 75098 75233 75254 75251 75252 75253 7503...,1840019440


In [3]:
# Drop the columns not required
city_df.drop(columns=["city_ascii","county_fips","lat","lng","population","density","source","military","incorporated","timezone","ranking","zips","id"],inplace=True)

print(city_df.shape)
city_df.sample(n=10)

(28372, 4)


,city,state_id,state_name,county_name
783,Encinitas,CA,California,San Diego
26196,Happys Inn,MT,Montana,Lincoln
5972,Nolanville,TX,Texas,Bell
12789,White Cloud,MI,Michigan,Newaygo
12007,Adrian,MO,Missouri,Bates
16027,West Wood,UT,Utah,Carbon
27687,Humboldt,MN,Minnesota,Kittson
1600,South Lake Tahoe,CA,California,El Dorado
24596,Corning,KS,Kansas,Nemaha
9210,Hudson,CO,Colorado,Weld


In [17]:
city_df_loc = city_df.loc[city_df["state_id"].isin('FL','CA')]
city_df_loc

TypeError: isin() takes 2 positional arguments but 3 were given

In [6]:
# Cities in FL only
city_df_fl = city_df.loc[city_df["state_id"] == 'FL']

print(city_df_fl.shape)
city_df_fl.head()

(911, 4)


,city,state_id,state_name,county_name
3,Miami,FL,Florida,Miami-Dade
16,Tampa,FL,Florida,Hillsborough
27,Orlando,FL,Florida,Orange
42,Jacksonville,FL,Florida,Duval
62,Sarasota,FL,Florida,Sarasota


In [7]:
# Cities in CA only
city_df_ca = city_df.loc[city_df["state_id"] == 'CA']

print(city_df_ca.shape)
city_df_ca.head()

(1470, 4)


,city,state_id,state_name,county_name
1,Los Angeles,CA,California,Los Angeles
12,San Francisco,CA,California,San Francisco
14,San Diego,CA,California,San Diego
20,Riverside,CA,California,Riverside
26,Sacramento,CA,California,Sacramento


In [13]:
# Merge CA and FL
city_df_ca_fl = city_df_fl
city_df_ca_fl = city_df_ca_fl.append(city_df_ca,ignore_index=True)
#df.append(df2, ignore_index=True)
city_df_ca_fl

,city,state_id,state_name,county_name
0,Miami,FL,Florida,Miami-Dade
1,Tampa,FL,Florida,Hillsborough
2,Orlando,FL,Florida,Orange
3,Jacksonville,FL,Florida,Duval
4,Sarasota,FL,Florida,Sarasota
...,...,...,...,...
2376,Valley Ranch,CA,California,Plumas
2377,Sugarloaf Village,CA,California,Tulare
2378,Cuyama,CA,California,Santa Barbara
2379,Cartago,CA,California,Inyo


In [14]:
#Export the data to CSV
file_path_export=Path("../../data/processed/city_county_mapping.csv")
city_df_ca_fl.to_csv(file_path_export,index=False)

In [15]:
import os,sys
parentdir = Path(os.path.abspath("../.."))
sys.path.insert(0,parentdir)

# Get the DB password
from config import db_password

from sqlalchemy import create_engine
import psycopg2
# import the psycopg2 database adapter for PostgreSQL
from psycopg2 import connect, extensions, sql

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT